In [1]:
import numpy as np
import pandas as pd
from list_of_words import *
from post_keyword_finder import find_words, doctors_identifier,trash_identifier

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('comments_likes_reposts_final1.csv')

In [39]:
df.head(1)

,Unnamed: 0,post_id,user_id,post,sex,Name_Surname,comments,platform,date,key_word,...,work_place,site,bdate,likes_count,comments_count,reposts_count,graduation_year,education_status,users_liked,users_reposted
0,0,1383.0,174740577,🌸23.02-16.00<br>🌸24.02-16.00<br>🌸26.02-10.00<b...,Female,Yulia_Baboshina,[],api_instagram,2018-02-19 17:08:06,#глаза,...,РГСУ,NaN,9.11,0,0,0,0.0,NaN,"{'count': 0, 'users': []}","{'count': 0, 'users': []}"


In [4]:
df_gr = df.groupby(['user_id','date','post']) # grouping for removing totally indeticall rows
df_gr = df_gr.nth(0).reset_index() #selecting first(one) of them

##  Filtering through Post, Activity, Interest, Work, Website,Faculty columns

In [ ]:
# this tab was not run, as it was taking too long time, performed in vk12
"""%%time
df_gr['post_num_keywords'] = df_gr.apply(lambda x: find_words(x,list_of_keywords_posts,'post','count'),axis=1)
df_gr['post_set_keywords'] = df_gr.apply(lambda x: find_words(x,list_of_keywords_posts,'post','set'),axis=1)"""

In [5]:
%%time
#filling null values in activities and interests columns
df_gr.activities.fillna('',inplace=True)
df_gr.interests.fillna('',inplace=True)

Wall time: 28.4 ms


In [6]:
%%time
#Searches in activities
#doctors
df_gr['act_num_keyword'] = df_gr.apply(lambda x: find_words(x,words_nohashtag,'activities','count'),axis=1)
df_gr['act_set_keyword'] = df_gr.apply(lambda x: find_words(x,words_nohashtag,'activities','set'),axis=1)

df_gr['act_oculist_num_keyword'] = df_gr.apply(lambda x: find_words(x,oculist_keyword_act,'activities','count'),axis=1)
df_gr['act_oculist_set_keyword'] = df_gr.apply(lambda x: find_words(x,oculist_keyword_act,'activities','set'),axis=1)

df_gr['act_kosmet_num_keyword'] = df_gr.apply(lambda x: find_words(x,kosmet_keyword_act,'activities','count'),axis=1)

#Searches in interests
df_gr['int_num_keyword'] = df_gr.apply(lambda x: find_words(x,words_nohashtag,'interests','count'),axis=1)
df_gr['int_set_keyword'] = df_gr.apply(lambda x: find_words(x,words_nohashtag,'interests','set'),axis=1)

df_gr['int_kosmet_num_keyword'] = df_gr.apply(lambda x: find_words(x,kosmet_keyword_int,'interests','count'),axis=1)
#this column: int_oculist_num_keyword  --> all the values are 0
#df_gr['int_oculist_num_keyword'] = df_gr.apply(lambda x: find_words(x,oculist_keyword_int,'interests','count'),axis=1)

Wall time: 1min 36s


In [7]:
#filling null values in work_place column
df_gr.work_place.fillna('',inplace=True)

In [8]:
%%time
#Searches in work place
#doctors
df_gr['work_doctors_num_keyword'] = df_gr.apply(lambda x: find_words(x,doctors_keyword_work,'work_place','count'),axis=1)
df_gr['work_doctors_set_keyword'] = df_gr.apply(lambda x: find_words(x,doctors_keyword_work,'work_place','set'),axis=1)

#oculist
df_gr['work_oculist_num_keyword'] = df_gr.apply(lambda x: find_words(x,oculist_keyword_work,'work_place','count'),axis=1)
df_gr['work_oculist_set_keyword'] = df_gr.apply(lambda x: find_words(x,oculist_keyword_work,'work_place','set'),axis=1)

#trash
df_gr['work_kosmet_num_keyword'] = df_gr.apply(lambda x: find_words(x,kosmet_keyword_work,'work_place','count'),axis=1)
df_gr['work_veter_num_keyword'] = df_gr.apply(lambda x: find_words(x,veter_keyword_work,'work_place','count'),axis=1)

Wall time: 28.2 s


In [9]:
#filling null values in faculty_name column
df_gr.faculty_name.fillna('',inplace=True)

In [10]:
%%time
#Searches in faculty_name
df_gr['faculty_med_num_keyword'] = df_gr.apply(lambda x: find_words(x,list_of_keywords_med_faculty,'faculty_name','count'),axis=1)
df_gr['faculty_med_set_keyword'] = df_gr.apply(lambda x: find_words(x,list_of_keywords_med_faculty,'faculty_name','set'),axis=1)

Wall time: 7.96 s


In [11]:
#filling null values in site(website) column
df_gr.site.fillna('',inplace=True)

In [12]:
%%time
#Searches in faculty_name
df_gr['web_isdoctors'] = df_gr.apply(lambda x: find_words(x,doctors_keyword_web,'site','exist'),axis=1)
df_gr['web_iskosmet'] = df_gr.apply(lambda x: find_words(x,kosmet_keyword_web,'site','exist'),axis=1)
df_gr['web_num_kosmet'] = df_gr.apply(lambda x: find_words(x,kosmet_keyword_web,'site','count'),axis=1)

Wall time: 13.4 s


In [13]:
%%time
#Searches in faculty_name
#df_gr['work_ispediator'] = df_gr.apply(lambda x: find_words(x,'педиатр','work_place','exist'),axis=1)


Wall time: 3.94 s


### End of the filtering

In [14]:
#columns based on criteria
df_gr['doctor'] = 0
df_gr['oculist'] = 0
df_gr['pediator'] = 0
df_gr['client'] = 0
df_gr['trash'] = 0

## Doctors identification

In [15]:
word_list_act = ['офтальмолог','астигматизм','глаукома','окулист','миопия']
doctors_identifier(df_gr, word_list_act, 'act_num_keyword', 'act_set_keyword', thresh=3, flag=0)
word_list_int = ['офтальмолог','астигматизм','глаукома','окулист','миопия']
doctors_identifier(df_gr, word_list_int, 'int_num_keyword', 'int_set_keyword', thresh=3, flag=0)

doctors_identifier(df_gr, oculist_keyword_act.split('|'), 'act_oculist_num_keyword', 'act_oculist_set_keyword', thresh=3, flag=0)

In [16]:
# taking care of all the rows which have list lenght = 1 and are note 'детск'
df_gr_work1 = df_gr[df_gr.work_doctors_num_keyword==1]
df_gr_detsk = df_gr_work1.work_doctors_set_keyword.apply(lambda x : x if 'детск' not in x[0].lower() else '-1')
df_gr_non_detsk_ind = df_gr_detsk[df_gr_detsk!='-1'].index
df_gr.loc[df_gr_non_detsk_ind,'doctor'] += 1

df_gr_work_ped = df_gr_detsk.apply(lambda x :'педиатр' in ' '.join(x).lower())
df_gr_work_ped_ind = df_gr_work_ped[df_gr_work_ped==True].index
df_gr.loc[df_gr_work_ped_ind,'pediator'] += 1

word_list_work = ['офтальмолог','астигматизм','глаукома','окулист','зрени','Глаз','опти','OPTIC']
doctors_identifier(df_gr, word_list_work, 'work_doctors_num_keyword', 'work_doctors_set_keyword', thresh=2, flag=1)

df_gr_ped = df_gr[df_gr['work_doctors_num_keyword']>=2]
df_gr_ped = df_gr_ped.work_doctors_set_keyword.apply(lambda x :'детск' in ' '.join(x).lower())
df_gr_ped_ind = df_gr_ped[df_gr_ped==True].index
df_gr.loc[df_gr_ped_ind,'pediator'] += 1


In [17]:
#faculty name
df_gr_fac = df_gr[df_gr['faculty_med_num_keyword']>=1]
df_gr_fac_ind = df_gr_fac.index
df_gr.loc[df_gr_fac_ind,'doctor'] += 1

df_gr_fac_ped = df_gr_fac.faculty_med_set_keyword.apply(lambda x :'педиатр' in ' '.join(x).lower())
df_gr_fac_ind = df_gr_fac_ped[df_gr_fac_ped==True].index
df_gr.loc[df_gr_fac_ind,'pediator'] += 1

In [18]:
# web

df_gr_web = df_gr[df_gr['web_isdoctors']>=1]
df_gr_web_ind = df_gr_web.index
df_gr.loc[df_gr_web_ind,'doctor'] += 1

In [19]:
doctors_identifier(df_gr, oculist_keyword_act.split('|'), 'act_oculist_num_keyword', 'act_oculist_set_keyword', thresh=3, flag=0)

### End of Doctors identification

## Trash identification

In [20]:
trash_identifier(df_gr, 'work_kosmet_num_keyword', thresh=2)

In [21]:
trash_identifier(df_gr, 'act_kosmet_num_keyword', thresh=2)

In [22]:
trash_identifier(df_gr, 'int_kosmet_num_keyword', thresh=2)

In [23]:
trash_identifier(df_gr, 'web_num_kosmet', thresh=2)

### End of Trash identification

In [ ]:
# ideally should be saved at this point but memory is not enoigh

In [ ]:
#df_gr.to_csv('user_info_filtered.csv', index=False)

# Dropping some columns to be able to save the .csv file

In [30]:
del df_gr['Unnamed: 0']

In [32]:
del df_gr['act_num_keyword']
del df_gr['act_set_keyword']
del df_gr['act_oculist_num_keyword']
del df_gr['act_oculist_set_keyword']
del df_gr['act_kosmet_num_keyword']
del df_gr['int_num_keyword']
del df_gr['int_set_keyword']
del df_gr['int_kosmet_num_keyword']

In [33]:
del df_gr['work_doctors_num_keyword']
del df_gr['work_doctors_set_keyword']
del df_gr['work_oculist_num_keyword']
del df_gr['work_oculist_set_keyword']
del df_gr['work_kosmet_num_keyword'] 
del df_gr['work_veter_num_keyword']

In [34]:
del df_gr['faculty_med_num_keyword']  
del df_gr['faculty_med_set_keyword'] 

In [35]:
del df_gr['web_isdoctors'] 
del df_gr['web_iskosmet']
del df_gr['web_num_kosmet']

In [36]:
del df_gr['work_ispediator']

In [37]:
df_gr.to_csv('user_info_filtered.csv', index=False)

In [33]:
(df_gr['trash']*(-1)*df_gr['doctor']).value_counts()

 0    249056
-2       207
-1       159
-3        40
-4        31
-6         7
-8         7
-5         3
dtype: int64

In [ ]:
# TRASHHHHH for sure user ids: 55293666, 52787032, 133080605, 153342056, 147080249, 187670366, 189097711, 190247060,204377934,
# 218436248, 241212180, 315643103, 332113299, 13121087, 

In [ ]:
# DOCTOR 45570

In [261]:
b =  df_gr_c['work_doctors_set_keyword'].apply(lambda x :'детск' in ' '.join(x).lower() )
b.value_counts()

In [60]:
c= df_gr['web_setkosmet'].apply(lambda x : str(x))
c.value_counts()[1:]

In [620]:
q=c.apply(lambda x :'глазного центра' in x.lower())
q[q==True].index

In [ ]:
#Вам предложить комания познакомьтесь с информацией  tri-z.ru